# Fedbiomed Researcher

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Create an experiment to train a model on the data found

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [2]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"
               ]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [3]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 4, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [4]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 5

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None,
                 tensorboard=True
                )

2022-03-21 11:23:26,981 fedbiomed INFO - Component environment:
2022-03-21 11:23:26,984 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-03-21 11:23:27,290 fedbiomed INFO - Messaging researcher_1fecd236-3507-4a58-9921-8d364492a6d1 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f79228be640>
2022-03-21 11:23:27,338 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-03-21 11:23:27,342 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Message received: {'researcher_id': 'researcher_1fecd236-3507-4a58-9921-8d364492a6d1', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-03-21 11:23:37,352 fedbiomed INFO - Node selected for training -> node_fa6a1655-e676-42a8-a6a5-fe2630057d46
2022-03-21 11:23:37,366 fedbiomed DEBUG - Model file has been saved: /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Experiment_0000/my_model_0dc3b90d-e9d3-4e1

Start tensorboard to see loss value after every iteration during training. It is normal to see empty screen. After you run the experiment you will be able to see the changes on the dashboard. Notebook will refresh results in every 30 seconds. You can also click refresh button to see current training steps. 

In [6]:
from fedbiomed.researcher.environ import environ
tensorboard_dir = environ['TENSORBOARD_RESULTS_DIR']

In [7]:
%load_ext tensorboard

In [8]:
tensorboard --logdir "$tensorboard_dir"

2022-03-21 11:30:54,064 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / CRITICAL - Node stopped in signal_handler, probably by user decision (Ctrl C)


In [5]:
exp.run()

2022-03-21 11:23:46,468 fedbiomed INFO - Sampled nodes in round 0 ['node_fa6a1655-e676-42a8-a6a5-fe2630057d46']
2022-03-21 11:23:46,469 fedbiomed INFO - Send message to node node_fa6a1655-e676-42a8-a6a5-fe2630057d46 - {'researcher_id': 'researcher_1fecd236-3507-4a58-9921-8d364492a6d1', 'job_id': 'f0e1e380-cd6c-40cf-b598-baf92a3128b2', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 4, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/21/my_model_0dc3b90d-e9d3-4e1f-91e1-8bcf61a4e315.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/21/aggregated_params_init_4ec61fab-28b1-4dc5-a65c-e7feb0663b7f.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_fa6a1655-e676-42a8-a6a5-fe2630057d46': ['dataset_9c41414b-1bf1-4783-a437-cbe7dec0ae78']}}
2022-03-21 11:23:46,469 fedbiomed DEBUG - researcher_1fecd236-3507-4a58-9921-8d364492a6d1
2022-03-21 11:23:46,485 fedbiomed INFO - log fr

2022-03-21 11:24:08,503 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 2 | Completed: 960/48000 (2%) 
 					 Loss: 0.220760 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:24:09,812 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 2 [1392/48000 (3%)]	Loss: 0.438379
2022-03-21 11:24:09,827 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 2 | Completed: 1440/48000 (3%) 
 					 Loss: 0.438379 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:24:11,274 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 2 [1872/48000 (4%)]	Loss: 0.135610
2022-03-21 11:24:11,286 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 2 | Completed: 1920/48000 (4%) 
 					 Loss: 0.135610 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:24:12,142 fedbiomed INFO - log from: 

2022-03-21 11:24:30,591 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 4 [2352/48000 (5%)]	Loss: 0.106322
2022-03-21 11:24:30,592 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 4 | Completed: 2400/48000 (5%) 
 					 Loss: 0.106322 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:24:31,750 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 4 [2832/48000 (6%)]	Loss: 0.309831
2022-03-21 11:24:31,754 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 4 | Completed: 2880/48000 (6%) 
 					 Loss: 0.309831 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:24:33,318 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 4 [3312/48000 (7%)]	Loss: 0.249106
2022-03-21 11:24:33,347 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 

2022-03-21 11:25:12,673 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 1 | Completed: 480/48000 (1%) 
 					 Loss: 0.140566 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:25:13,404 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 1 [912/48000 (2%)]	Loss: 0.337855
2022-03-21 11:25:13,405 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 1 | Completed: 960/48000 (2%) 
 					 Loss: 0.337855 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:25:14,180 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 1 [1392/48000 (3%)]	Loss: 0.070607
2022-03-21 11:25:14,181 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 1 | Completed: 1440/48000 (3%) 
 					 Loss: 0.070607 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:25:14,917 fedbiomed INFO - log from: no

2022-03-21 11:25:30,572 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 3 [1872/48000 (4%)]	Loss: 0.122315
2022-03-21 11:25:30,574 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 3 | Completed: 1920/48000 (4%) 
 					 Loss: 0.122315 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:25:31,380 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 3 [2352/48000 (5%)]	Loss: 0.117146
2022-03-21 11:25:31,381 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 3 | Completed: 2400/48000 (5%) 
 					 Loss: 0.117146 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:25:32,179 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 3 [2832/48000 (6%)]	Loss: 0.085296
2022-03-21 11:25:32,181 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 

...

2022-03-21 11:27:49,014 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 3 | Completed: 1920/48000 (4%) 
 					 Loss: 0.112503 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:27:50,445 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 3 [2352/48000 (5%)]	Loss: 0.061857
2022-03-21 11:27:50,449 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 3 | Completed: 2400/48000 (5%) 
 					 Loss: 0.061857 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:27:52,161 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 3 [2832/48000 (6%)]	Loss: 0.023454
2022-03-21 11:27:52,164 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 3 | Completed: 2880/48000 (6%) 
 					 Loss: 0.023454 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:27:53,599 fedbiomed INFO - log from:

2022-03-21 11:28:33,539 fedbiomed DEBUG - researcher_1fecd236-3507-4a58-9921-8d364492a6d1
2022-03-21 11:28:33,551 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Message received: {'researcher_id': 'researcher_1fecd236-3507-4a58-9921-8d364492a6d1', 'job_id': 'f0e1e380-cd6c-40cf-b598-baf92a3128b2', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 4, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/21/my_model_0dc3b90d-e9d3-4e1f-91e1-8bcf61a4e315.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/21/aggregated_params_efab6f3c-44d5-42e2-ab70-968c2667cac4.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_fa6a1655-e676-42a8-a6a5-fe2630057d46': ['dataset_9c41414b-1bf1-4783-a437-cbe7dec0ae78']}}
2022-03-21 11:28:33,558 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'research

2022-03-21 11:28:54,060 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 2 | Completed: 1920/48000 (4%) 
 					 Loss: 0.025598 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:28:54,848 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 2 [2352/48000 (5%)]	Loss: 0.030363
2022-03-21 11:28:54,850 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 2 | Completed: 2400/48000 (5%) 
 					 Loss: 0.030363 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:28:55,603 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 2 [2832/48000 (6%)]	Loss: 0.006776
2022-03-21 11:28:55,604 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 2 | Completed: 2880/48000 (6%) 
 					 Loss: 0.006776 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:28:56,366 fedbiomed INFO - log from:

2022-03-21 11:29:13,700 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 4 [3312/48000 (7%)]	Loss: 0.005236
2022-03-21 11:29:13,702 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 4 | Completed: 3360/48000 (7%) 
 					 Loss: 0.005236 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:29:14,416 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 4 [3792/48000 (8%)]	Loss: 0.014801
2022-03-21 11:29:14,417 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 Epoch: 4 | Completed: 3840/48000 (8%) 
 					 Loss: 0.014801 
					 Boss: 12.000000 
					 ---------
2022-03-21 11:29:15,215 fedbiomed INFO - log from: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 / DEBUG - Train Epoch: 4 [4272/48000 (9%)]	Loss: 0.039834
2022-03-21 11:29:15,225 fedbiomed INFO - TRAINING 
					 NODE_ID: node_fa6a1655-e676-42a8-a6a5-fe2630057d46 
					 

5

To display current values please click refresh button on the TensorBoard screen

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()

Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())


## Optional : searching the data

In [ ]:
from fedbiomed.researcher.requests import Requests

r = Requests()
data = r.search(tags)

import pandas as pd
for node_id in data.keys():
    print('\n','Data for ', node_id, '\n\n', pd.DataFrame(data[node_id]))

Feel free to try your own models :D